Permutation of both DMN nodes

In [1]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_style("darkgrid")
from scipy import stats

from bct.algorithms import degrees_und, community_louvain, distance_bin
from tvb.datatypes import connectivity # this breaks the print statement

In [2]:
def exclude_input_nodes_dmn(mylist, idx):
    # node 15 is included in the analysis
    input_nodes = [20, 21, 23]
    # check if array has the dimension with all nodes
    all_nodes = range(33) # range excludes the last value
    assert len(all_nodes) == len(mylist)
    # also delete the self-entry
    input_nodes.append(idx)
    
    return np.delete(mylist, input_nodes) 

In [3]:
path = '../Output/dynamic_analysis/with_behaviour/with_homeostasis/dmn_permutation/dynamic_analysis_dmn_permutation.pickle'
with open(path, 'rb') as fh:
    dmn_permutation = pickle.load(fh)

# get only the nodes of interes
all_nodes = range(33)
input_nodes = [20, 21, 23]
nodes = np.delete(all_nodes, input_nodes)

In [4]:
# load connectivity matrix
conn = connectivity.Connectivity.from_file(source_file="connectivity_66.zip")
# returns the numbers of edges connected to each node
deg = degrees_und(conn.weights)

# community
communities, q = community_louvain(conn.weights, gamma=1, seed=42)
# get the number of communities and the number of nodes in each community
n_communities = np.unique(communities)
# Note: The first element is being ignored because n_communities starts from 1 and there are no module 0
n_node_each_community = np.bincount(communities)[1:]
# get the shortest path length between pairs of nodes
path_length = distance_bin(conn.weights)

In [6]:
path_length.shape
path_length

array([[ 0.,  3.,  2., ...,  3.,  3.,  3.],
       [ 3.,  0.,  2., ...,  2.,  3.,  2.],
       [ 2.,  2.,  0., ...,  2.,  3.,  2.],
       ..., 
       [ 3.,  2.,  2., ...,  0.,  2.,  1.],
       [ 3.,  3.,  3., ...,  2.,  0.,  2.],
       [ 3.,  2.,  2., ...,  1.,  2.,  0.]])

In [ ]:
dmn_permutation[1][2].keys()

In [ ]:
dmn_permutation[1][5]['correlation_ws_single_nodes']

In [ ]:
len(nodes)

In [ ]:
nodes

In [ ]:
dmn_permutation[0][2]['correlation_ws_single_nodes'].shape

In [ ]:
import pdb
results = {}

# node1 corresponds to the TN node

for node1 in nodes:
    results[node1] = {}
    corr1 = []
    distanc1 = exclude_input_nodes_dmn(path_length[node1, :33], node1)
    # get the correlation_ws_mean node values for the first node
    # node2 is the TP node
    for node2 in nodes:
        if not node2 == node1: 
            corr1.append(dmn_permutation[node1][node2]['correlation_ws_mean_nodes'])
    results[node1]['t'], results[node1]['p'] = stats.pearsonr(corr1, distanc1)

In [ ]:
results

## Mean activity over time

In [ ]:
# TODO: There is one change on the python file called grid_search.py to save the mean of ws over each node.
# It has however, not been calculated yet. I will see if I need to do it or not.
# TODO: You will also need to adapt this code snipet to calculate what you want
import pdb
results_t = np.zeros((len(nodes), len(nodes)))
results_p = np.zeros((len(nodes), len(nodes)))
# node1 corresponds to the TN node
for node1 in nodes:
    distanc1 = exclude_input_nodes_dmn(path_length[node1, :33], node1)
    # get the correlation_ws_mean node values for the first node
    # node2 is the TP node
    corr1 = [dmn_permutation[node1][node2]['correlation_ws_mean_nodes'] for node2 in nodes if not node2==node1]
    
    results_t[node1][node2], results_p[node1][node2] = stats.pearsonr(corr1, distanc1)

In [ ]:
dmn_permutation[0][2]['mean_activity_over_time'].shape

In [ ]:
corr1

In [ ]:
distanc1